## 数据集划分

In [1]:
# 导入必要的库
import os
import pandas as pd
import numpy as np
# 检查当前工作目录
current_directory = os.getcwd()
print("当前工作目录:", current_directory)

d:\anaconda\envs\usual\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda\envs\usual\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\anaconda\envs\usual\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


当前工作目录: d:\Issue\2024_2025\夏雨婷文章\md06\xyt\md05\02分子指纹


In [2]:
# imblearn是处理不平衡数据集的库，包括过采样和欠采样方法
from imblearn.datasets import make_imbalance          
from imblearn.over_sampling import KMeansSMOTE,SMOTE,SVMSMOTE
from imblearn.under_sampling import AllKNN,CondensedNearestNeighbour,NearMiss
from imblearn.combine import SMOTEENN,SMOTETomek
# sklearn提供机器学习模型和数据处理工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import average_precision_score
# 配置Jupyter Notebook环境
# 设置高分辨率图像输出
%config InlineBackend.figure_format='retina'
# 设置matplotlib图形直接在Jupyter Notebook中显示
%matplotlib inline

In [3]:
# 导入os模块和计数器
import os
from collections import Counter
# 导入RDKit模块，用于化学信息学处理
import rdkit as rd
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import RDConfig
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem.rdMolDescriptors import GetAtomPairFingerprint
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.AtomPairs import Pairs, Torsions
# 导入更多机器学习和模型评估工具
import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# 导入绘图库
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 导入数据处理和模型选择工具
from sklearn.utils import shuffle 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split,cross_validate, StratifiedKFold
from sklearn.utils  import shuffle
import joblib

In [7]:
import os
os.chdir("D://Issue/2024_2025/夏雨婷文章/md05/xyt/md04/2-分子指纹/指纹表征/version-1/dataset")

In [8]:
# 读取并划分数据集
data = pd.read_csv('bioactivity_preprocessed_MGAM.csv')
x = data["canonical_smiles"]  # SMILES字符串作为特征
y = data["bioactivity_class"]    # 生物活性类别作为目标变量
#x_mor_train和x_mor_test分别为训练集和测试集中的特征数据，y_mor_train和y_mor_test分别为训练集和测试集中的目标变量数据。
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=13)
#test_size=0.2: 测试集大小占总数据的20%
#stratify=y: 划分时保持y中的类别比例，确保训练集和测试集中各类别的比例与原始数据集相同
#random_state=13：设置随机数种子以确保结果的可重复性

In [10]:
# 打印训练集和测试集的维度，确认划分正确
print("训练集特征维度:", x_train.shape)
print("训练集目标维度:", y_train.shape)
print("测试集特征维度:", x_test.shape)
print("测试集目标维度:", y_test.shape)

训练集特征维度: (605,)
训练集目标维度: (605,)
测试集特征维度: (152,)
测试集目标维度: (152,)


In [11]:
# 将划分的数据集保存为CSV文件
x_train.to_csv('train_features.csv', index=False)  # 保存训练集特征
y_train.to_csv('train_target.csv', index=False)  # 保存训练集目标变量
x_test.to_csv('test_features.csv', index=False)  # 保存测试集特征
y_test.to_csv('test_target.csv', index=False)  # 保存测试集目标变量

In [12]:
# 将特征和目标变量合并后再次保存，方便后续处理
train_data = pd.concat([x_train, y_train], axis=1)
train_data.to_csv('train_dataset.csv', index=False)  # 保存训练数据集
test_data = pd.concat([x_test, y_test], axis=1)
test_data.to_csv('test_dataset.csv', index=False)  # 保存测试数据集

## 分子指纹表征

In [13]:
# 读取测试数据集
ar2 = pd.read_csv('test_dataset.csv')

# 初始化一个列表来存储Canonical SMILES
Canonical_SMILES = []
# 遍历数据集中的canonical_smiles列，并将其值添加到列表中
for i in ar2.canonical_smiles:
    Canonical_SMILES.append(i)

# 将Canonical SMILES列表复制给新变量
ar1_smiles = Canonical_SMILES

# 初始化一个列表来存储规范化后的SMILES字符串
c_smiles = []
# 遍历SMILES字符串
for ds in ar1_smiles:
    try:
         # 尝试使用RDKit的Chem.CanonSmiles函数规范化SMILES字符串
        cs = Chem.CanonSmiles(ds)
        # 如果成功，添加到列表中
        c_smiles.append(cs)
    except:
        # 如果规范化失败，打印失败的SMILES字符串
        print('Invalid SMILES:', ds)

# 将规范化的SMILES字符串转换为RDKit分子对象
ms = [Chem.MolFromSmiles(x) for x in c_smiles]

### 扩展连通性指纹 (Extended Connectivity FingerPrints, ECFPs)

In [14]:
# 为每个分子生成Morgan指纹，指定半径为2，位数为2048
Morganfps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=2048) for x in ms]

# 将生成的Morgan指纹写入到文件中
with open('morganfps.txt','a') as c:
    for i in range(len(Morganfps)):
        # 将指纹转换为位字符串
        str1 = Morganfps[i].ToBitString()
        # 遍历位字符串，写入每一位，并用逗号分隔
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹结束后换行    
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的Morgan指纹文件   
ecfp4 = open("morganfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件内容
for line in ecfp4.readlines():
    # 按逗号分隔每一行，转换成列表
    line=line.split(',')
    # 添加到列表中
    MF_ecfp4.append(line)
# 关闭文件 
ecfp4.close() 

# 将读取的数据转换为DataFrame，并去掉最后一个空列（因为每行末尾有一个逗号）
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]
# 将这两列与Morgan指纹的DataFrame合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)
# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-ecfps.csv',index=False)

In [16]:
# 为每个分子生成Morgan指纹，指定半径为2，位数为2048
Morganfps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=2048) for x in ms]

# 将生成的Morgan指纹写入到文件中
with open('morganfps.txt','a') as c:
    for i in range(len(Morganfps)):
        # 将指纹转换为位字符串
        str1 = Morganfps[i].ToBitString()
        # 遍历位字符串，写入每一位，并用逗号分隔
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹结束后换行    
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的Morgan指纹文件   
ecfp4 = open("morganfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件内容
for line in ecfp4.readlines():
    # 按逗号分隔每一行，转换成列表
    line=line.split(',')
    # 添加到列表中
    MF_ecfp4.append(line)
# 关闭文件 
ecfp4.close() 

# 将读取的数据转换为DataFrame，并去掉最后一个空列（因为每行末尾有一个逗号）
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]
# 将这两列与Morgan指纹的DataFrame合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)
# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-ecfps.csv',index=False)

### 功能类圆形指纹（Functional-Class Fingerprints, FCFPs）

In [17]:
# 生成分子的FCFPs（功能类圆形指纹），半径为2，位数为2048，使用特征（useFeatures=True）
FCFPs = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=2048, useFeatures=True) for x in ms]

# 将生成的FCFPs写入到txt文件中
with open('fcfps.txt','a') as c:
    for i in range(len(FCFPs)):
        # 将每个指纹转换为位字符串
        str1 = FCFPs[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的FCFPs文件
ecfp4 = open("fcfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 按逗号分割每行，转换成列表
    line=line.split(',')
    # 将分割后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件  
ecfp4.close() 

# 将读取的数据转换为DataFrame，并删除最后一个空列（因为每行末尾有多余的逗号）
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与FCFPs数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-fcfps.csv',index=False)

### Avalon指纹

In [18]:
# 生成Avalon指纹：对ms列表中的每个分子对象x生成Avalon指纹
Avalonfps = [pyAvalonTools.GetAvalonFP(x) for x in ms]

# 打开（或创建）'avalonfps.txt'文件以追加模式
with open('avalonfps.txt','a') as c:
    # 遍历所有生成的Avalon指纹
    for i in range(len(Avalonfps)):
        # 将指纹转换为位字符串
        str1 = Avalonfps[i].ToBitString()
         # 遍历位字符串中的每一位，并将其写入文件，每位之间用逗号分隔
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取'avalonfps.txt'文件
ecfp4 = open("avalonfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close()

# 将读取的指纹数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集ar2中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与Avalon指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件'MGAM-test-avalonfps.csv'
ar3.to_csv('MGAM-test-avalonfps.csv',index=False)

### MACCS指纹

In [19]:
# 为列表中的每个分子生成MACCS指纹
MACCS = [AllChem.GetMACCSKeysFingerprint(x) for x in ms]

# 打开（或创建）'maccs.txt'文件以追加模式写入MACCS指纹数据
with open('maccs.txt','a') as c:
    for i in range(len(MACCS)):
         # 将每个指纹转换为位字符串
        str1 = MACCS[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件写入完成后自动关闭
    c.close()

# 读取保存的'maccs.txt'文件
ecfp4 = open("maccs.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close() 

# 将读取的数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与MACCS指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-maccs.csv',index=False)

### RDKit指纹

In [20]:
# 为ms列表中的每个分子生成RDK指纹，设置最大路径长度为5，指纹大小为2048位
RDKfps = [Chem.RDKFingerprint(x,maxPath=5,fpSize=2048) for x in ms]

# 打开（或创建）'rdkfps.txt'文件，以追加模式写入RDK指纹数据
with open('rdkfps.txt','a') as c:
    for i in range(len(RDKfps)):
        # 将每个指纹转换为位字符串
        str1 = RDKfps[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的'rdkfps.txt'文件
ecfp4 = open("rdkfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close() 

# 将读取的指纹数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与RDK指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-rdkfps.csv',index=False)

### 生成散列化的原子对（Atom Pair, AP）指纹

In [22]:
from rdkit.Chem import rdMolDescriptors
# 为ms列表中的每个分子生成散列化的原子对指纹
APfps = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(x) for x in ms]

# 打开（或创建）'apfps.txt'文件，以追加模式写入散列化的原子对指纹数据
with open('apfps.txt','a') as c:
    # 遍历所有生成的散列化的原子对指纹
    for i in range(len(APfps)):
        # 将每个指纹转换为位字符串
        str1 = APfps[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的'apfps.txt'文件
ecfp4 = open("apfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close() 

# 将读取的指纹数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与散列化的原子对指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-apfps.csv',index=False)

### 生成散列化的拓扑扭转（Topological Torsion）指纹

In [24]:
# 为ms列表中的每个分子生成散列化的拓扑扭转（Topological Torsion）指纹
Topfps = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(x) for x in ms]

# 打开（或创建）'topfps.txt'文件，以追加模式写入拓扑扭转指纹数据
with open('topfps.txt','a') as c:
    # 遍历所有生成的拓扑扭转指纹
    for i in range(len(Topfps)):#将RDKfps修改为Topfps
        # 将每个指纹转换为位字符串
        str1 = RDKfps[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的'topfps.txt'文件
ecfp4 = open("topfps.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close() 

# 将读取的指纹数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与拓扑扭转指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-topfps.csv',index=False)

### 药效团（Pharmacophore）指纹

In [38]:
# 导入RDKit的药效团指纹模块
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D, Generate

# 为列表中的每个分子生成药效团指纹
feats = [Generate.Gen2DFingerprint(x, Gobbi_Pharm2D.factory) for x in ms]

# 打开（或创建）'feats.txt'文件以追加模式写入药效团指纹数据
with open('feats.txt','a') as c:
    for i in range(len(feats)):
        # 将每个指纹转换为位字符串
        str1 = feats[i].ToBitString()
        # 遍历位字符串，并将每一位后跟逗号写入文件
        for j in str1:
            c.write(str(int(j))+',')
        # 每个指纹的位字符串结束后换行
        c.write('\n')
    # 文件操作完成后自动关闭
    c.close()

# 读取保存的'feats.txt'文件
ecfp4 = open("feats.txt","r",encoding='utf-8') 
MF_ecfp4=[]
# 逐行读取文件中的内容
for line in ecfp4.readlines():
    # 将每行按逗号分割，转换成列表
    line=line.split(',')
    # 将处理后的列表添加到MF_ecfp4列表中
    MF_ecfp4.append(line)
# 关闭文件
ecfp4.close() 

# 将读取的指纹数据转换为DataFrame
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
# 删除DataFrame中最后一个空列，这个空列是由于每行结尾的逗号导致的
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

# 从原始数据集中提取canonical_smiles和bioactivity_class列
Y = ar2[["canonical_smiles","bioactivity_class"]]

# 将提取的列与药效团指纹数据合并
ar3 = pd.concat([Y,MF_ecfp4], axis=1)

# 将合并后的数据保存到新的CSV文件
ar3.to_csv('MGAM-test-feats.csv',index=False)